<a href="https://colab.research.google.com/github/31indianaSKKU/NBA_CareerLength/blob/main/Draft_History_NBA_com_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

고수님 답변

* 테이블이 두 페이지의 걸쳐 있으나 두 번째 페이지의 테이블을 가져오지 못함
-> 버튼을 잡지않고 svg를 잡는 문제가 있었습니다.
-> html의 기능으로 click을 구현하지 않고, javascript로 구현 되어있어 .click() 이 동작하지 않습니다.

* browser = webdriver.Chrome(service=service)부분에서 에러가 발생하는 부분은 쥬피터에서는 문제가 없는데 코랩에서는 에러가 나네요.
-> colab 환경은 화면이 존재하지 않는, linux환경입니다.
-> 일반적인 chrome은 동작하지 않기에, chromium을 사용하고, headless 모드로 사용해야합니다.

* 기타 문제점
-> 루프문에 문제가 있어서, 무한으로 동작을 하는 문제가 있었습니다.

In [1]:
# 이 부분은 처음 한번만 실행하면 됌.
# "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver


# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.T99eY8tL5z/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.w8jMVpvFMc/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.cyojnzDbK0/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [2]:
# 셀레니움 module import 
# from selenium import webdriver
# 2023.05.08 selenium webdriver 사용방법 변경 
from selenium.webdriver.common.by import By
from selenium import webdriver
import time 
from time import sleep
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import requests
import re
import os
import csv

# Dataframe 
import pandas as pd

class Draft_history:
    def __init__(self, name, team, affiliation, year, round_num, pick_num, overall_num):
        self.name = name
        self.team = team
        self.affiliation = affiliation
        self.year = year
        self.round_num = round_num
        self.pick_num = pick_num
        self.overall_num = overall_num

    def to_list(self):
        return [self.name, self.team, self.affiliation, self.year, self.round_num, self.pick_num, self.overall_num]
##################################################################################################################################
##################################################################################################################################
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('window-size=1080,3840')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('disable-dev-shm-usage')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
browser = webdriver.Chrome('chromedriver', options=chrome_options)
##################################################################################################################################
##################################################################################################################################


for yrs in range(2015, 2017):
    browser.implicitly_wait(10)
    browser.get(f"https://www.nba.com/stats/draft/history?Season={yrs}")

    draft_list = []
    while True:
        try:
##################################################################################################################################
##################################################################################################################################
            browser.implicitly_wait(5)
            
            next_button = browser.find_elements(By.CLASS_NAME, 'Pagination_button__sqGoH')[1]

            tbody = browser.find_element(By.CLASS_NAME, 'Crom_body__UYOcU')
            trs = tbody.find_elements(By.TAG_NAME, 'tr')

            for tr in trs:
                tds = tr.find_elements(By.TAG_NAME, 'td')
                if len(tds) == 0:
                    continue
                else:
                    name = tds[0].text
                    team = tds[1].text
                    affiliation = tds[2].text
                    year = tds[3].text
                    round_num = tds[4].text
                    pick_num = tds[5].text
                    overall_num = tds[6].text

                    draft = Draft_history(name, team, affiliation, year, round_num, pick_num, overall_num)
                    draft_list.append(draft.to_list())

            has_next = next_button.is_enabled()
            if(has_next == False):
                print("end page")
                break

            browser.execute_script("arguments[0].click();", next_button)
            
        except Exception as e:
            print(e)

    # 파일에 데이터를 저장합니다.
    file_name = f'draft_list_{yrs}.csv'
    with open(file_name, 'w', encoding="utf-8", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Name', 'Team', 'Affiliation', 'Year', 'Round', 'Pick', 'Overall'])
        
        for draft in draft_list:
            print(draft)
            writer.writerow(draft)

# Chrome 드라이버를 종료합니다.
browser.quit()


end page
['Karl-Anthony Towns', 'Minnesota Timberwolves', 'Kentucky', '2015', '1', '1', '1']
["D'Angelo Russell", 'Los Angeles Lakers', 'Ohio State', '2015', '1', '2', '2']
['Jahlil Okafor', 'Philadelphia 76ers', 'Duke', '2015', '1', '3', '3']
['Kristaps Porzingis', 'New York Knicks', 'CB Sevilla (Spain)', '2015', '1', '4', '4']
['Mario Hezonja', 'Orlando Magic', 'FC Barcelona Basquet (Spain)', '2015', '1', '5', '5']
['Willie Cauley-Stein', 'Sacramento Kings', 'Kentucky', '2015', '1', '6', '6']
['Emmanuel Mudiay', 'Denver Nuggets', 'Guangdong Southern Tigers (China)', '2015', '1', '7', '7']
['Stanley Johnson', 'Detroit Pistons', 'Arizona', '2015', '1', '8', '8']
['Frank Kaminsky', 'Charlotte Hornets', 'Wisconsin', '2015', '1', '9', '9']
['Justise Winslow', 'Miami Heat', 'Duke', '2015', '1', '10', '10']
['Myles Turner', 'Indiana Pacers', 'Texas', '2015', '1', '11', '11']
['Trey Lyles', 'Utah Jazz', 'Kentucky', '2015', '1', '12', '12']
['Devin Booker', 'Phoenix Suns', 'Kentucky', '2015',

질문1.
1. 누구나 접속 가능한 웹페이지.https://www.nba.com/stats/draft/history?Season=2022

2. 목적: 페이지 내의 테이블 데이터 수집.

3. 문제상황: 테이블이 두 페이지의 걸쳐 있으나 두 번째 페이지의 테이블을  가져오지 못함.그리고 현재 browser = webdriver.Chrome(service=service)부분에서 에러가 발생하는 부분은 쥬피터에서는 문제가 없는데 코랩에서는 에러가 나네요.


In [3]:
#질문했던 코드_내가 한 최종 코드_수정 전 코드



import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager

class Draft_history:
    def __init__(self, name, team, affiliation, year, round_num, pick_num, overall_num):
        self.name = name
        self.team = team
        self.affiliation = affiliation
        self.year = year
        self.round_num = round_num
        self.pick_num = pick_num
        self.overall_num = overall_num

    def to_list(self):
        return [self.name, self.team, self.affiliation, self.year, self.round_num, self.pick_num, self.overall_num]

# Chrome 드라이버를 시작합니다.
service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service)

for yrs in range(2015, 2017):
    browser.get(f"https://www.nba.com/stats/draft/history?Season={yrs}")
    browser.implicitly_wait(1)

    draft_list = []
    while True:
        try:
            table = browser.find_element(By.XPATH, '//*[@id="__next"]')
            trs = table.find_elements(By.XPATH, './/tr')

            for tr in trs:
                try:
                    tds = tr.find_elements(By.XPATH, './/td')
                    if len(tds) == 0:
                        continue
                    else:
                        name = tds[0].text
                        team = tds[1].text
                        affiliation = tds[2].text
                        year = tds[3].text
                        round_num = tds[4].text
                        pick_num = tds[5].text
                        overall_num = tds[6].text

                        draft = Draft_history(name, team, affiliation, year, round_num, pick_num, overall_num)
                        draft_list.append(draft.to_list())
                except StaleElementReferenceException:
                    continue

############################################################################################################################################################################### 
###############################################################################################################################################################################
###############################################################################################################################################################################
###############################################################################################################################################################################  
#문제 부분        
            #next_button = browser.find_element(By.XPATH, '//a[@title="Next page"]')
            next_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[5]/button[2]/svg')
            ###
            ### 
            ###
            next_button.click()
            browser.implicitly_wait(5)
###############################################################################################################################################################################
###############################################################################################################################################################################
###############################################################################################################################################################################
###############################################################################################################################################################################
            # 다음 페이지의 table과 trs를 업데이트합니다.
            table = browser.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table')
            trs = table.find_elements(By.XPATH, './/tr')
            for tr in trs:
                try:
                    tds = tr.find_elements(By.XPATH, './/td')
                    if len(tds) == 0:
                        continue
                    else:
                        name = tds[0].text
                        team = tds[1].text
                        affiliation = tds[2].text
                        year = tds[3].text
                        round_num = tds[4].text
                        pick_num = tds[5].text
                        overall_num = tds[6].text

                        draft = Draft_history(name, team, affiliation, year, round_num, pick_num, overall_num)
                        draft_list.append(draft.to_list())
                except StaleElementReferenceException:
                    continue
        
        except:
            break

 

    # 파일에 데이터를 저장합니다.
    file_name = f'draft_list_{yrs}.csv'
    with open(file_name, 'w', encoding="utf-8", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Name', 'Team', 'Affiliation', 'Year', 'Round', 'Pick', 'Overall'])
        for draft in draft_list:
            writer.writerow(draft)

# Chrome 드라이버를 종료합니다.
browser.quit()



ModuleNotFoundError: ignored